In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

# 아이디어

In [2]:
# AI 에이전트를 사용하여 연구 포스터 이미지 내의 다양한 그래프를 자동을 탐지하고 인식한 뒤 내용을 통합적으로 분석하는 시스템 개발

핵심 구성 요소

1. 포스터 이미지 입력

- 전체 연구 포스터를 하나의 이미지 파일로 입력받음

2. 시각 요소 탐지

- 포스터 내에 포함된 그래프, 표, 도식, 텍스트를 각각 영역별로 분할

- → 이를 위해 object detection 모델 (e.g. YOLO, DETR) 혹은 OCR 기반 레이아웃 분석 (e.g. LayoutLMv3) 활용 가능

3. 그래프 종류 및 내용 인식

- 그래프가 선형인지 막대형인지 등 종류를 분류

- 그래프 이미지에서 축 정보, 범례, 데이터 포인트 추출

- → computer vision + OCR 기반 그래프 파싱 (e.g. PlotQA, ChartOCR, DePlot 등 사용 가능)

4. 텍스트와의 연관 분석

- 그래프 주변 텍스트(캡션, 설명 등)와 연결하여 의미 해석

- → 텍스트-이미지 alignment 필요 (e.g. multimodal transformer or cross attention)

5. 통합 분석 및 요약

- 포스터 내 모든 그래프와 텍스트 정보를 연결하여 연구 주제나 결론 도출

- → 멀티에이전트 구조로 각 파트별 전문 AI를 둘 수도 있음

# 멀티에이전트 설계아이디어
| 에이전트 이름          | 역할                                  |
| ---------------- | ----------------------------------- |
| **레이아웃 인식 에이전트** | 포스터 내 시각적 영역 분할 (텍스트 vs 그래프 vs 표 등) |
| **그래프 인식 에이전트**  | 그래프 이미지 종류 분류 및 데이터 추출              |
| **텍스트 연관 에이전트**  | 그래프와 주변 설명 텍스트 매핑                   |
| **분석 요약 에이전트**   | 그래프 간 관계 분석 및 전체 요약 생성              |


# 스택

- 그래프 탐지 및 인식: DePlot, ChartOCR, PlotQA

- 텍스트 인식: Tesseract OCR, PaddleOCR, LayoutLMv3

- 멀티모달 분석: BLIP-2, GPT-4V, Kosmos-2, LLaVA

- 멀티 에이전트 프레임워크: LangGraph, CrewAI, Autogen

# Demo Implementation

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm

## Layout Detect Agent

In [5]:
def detect_layout():
    return None

In [4]:
def get_layout(image_path: str) -> dict:
    """Analyze a research poster image and return layout components such as text, graphs, and tables."""
    return detect_layout(image_path)

In [ ]:
handoff_to_graph = create_handoff_tool(
    agent_name="GraphAnalysisAgent",
    description="If the image contains one or more graphs that require interpretation or chart data extraction, hand over to GraphAnalysisAgent."
)

handoff_to_table = create_handoff_tool(
    agent_name="TableUnderstandingAgent",
    description="If the image contains tables that need parsing into structured format or statistical summary, hand over to TableUnderstandingAgent."
)

layout_agent = create_react_agent(
    model,
    tools=[layout_analysis, handoff_to_graph, handoff_to_table],
    prompt="""
You are a layout detection agent. Your job is to identify layout components (text, chart, table) from a research poster image.
If any graph needs interpretation, hand over to GraphAnalysisAgent.
If a table needs parsing or numerical analysis, hand over to TableUnderstandingAgent.
""",
    name="LayoutAgent"
)

## Graph Analysis Agent

In [6]:
def analyze_chart(image_path: str) -> dict:
    """
    주어진 그래프 이미지에서 데이터 포인트, 축, 레이블 등을 추출하여 반환.
    (DePlot 또는 ChartOCR 활용)
    """
    # 예시는 DePlot 기반의 placeholder
    return {
        "type": "bar",
        "x_axis": ["Q1", "Q2", "Q3", "Q4"],
        "y_axis_label": "Sales (in thousands)",
        "data": [120, 100, 80, 130],
        "summary": "Sales peaked in Q4, followed by Q1. Lowest in Q3."
    }

In [7]:
from langchain.tools import tool

@tool
def graph_analysis_tool(image_path: str) -> dict:
    """Analyze the chart in the image and return data points, chart type, and summary."""
    return analyze_chart(image_path)

In [ ]:
graph_agent = create_react_agent(
    model,
    tools=[graph_analysis_tool],
    prompt="""
You are a Graph Analysis Agent. Your job is to interpret a graph image and extract the following:
- Graph type (bar, line, pie, etc.)
- X and Y axis labels
- Data points
- A natural language summary of the chart

Use the tool `graph_analysis_tool` to analyze the graph and return a structured result.
""",
    name="GraphAnalysisAgent"
)

## Table Analysis Agent

In [ ]:
import pytesseract
import cv2

def parse_table(image_path: str) -> dict:
    """
    이미지에서 테이블을 OCR 기반으로 구조화하여 파싱
    """
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # OCR 수행
    raw_text = pytesseract.image_to_string(gray, config="--psm 6")
    
    # 예시 파싱 (줄 기준으로 단순 split)
    lines = raw_text.strip().split('\n')
    headers = lines[0].split()
    rows = [dict(zip(headers, line.split())) for line in lines[1:] if len(line.split()) == len(headers)]
    
    return {
        "headers": headers,
        "rows": rows,
        "summary": f"Parsed {len(rows)} rows. Columns: {headers}"
    }

In [ ]:
def table_analysis_tool(image_path: str) -> dict:
    """Analyze a table image and return structured data and summary statistics."""
    return parse_table(image_path)



In [ ]:
table_agent = create_react_agent(
    model,
    tools=[table_analysis_tool],
    prompt="""
You are a Table Understanding Agent. Your job is to:
- Interpret a table image
- Return structured rows and columns
- Provide a natural language summary such as statistics, patterns, or outliers

Use the tool `table_analysis_tool` to analyze the image.
""",
    name="TableUnderstandingAgent"
)

# Text Linking Agent

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")

def link_text_to_graph(graph_summary: str, text_blocks: list[str]) -> dict:
    """
    그래프 설명 요약과 텍스트 블록 간 연관성 분석
    """
    graph_emb = model.encode(graph_summary, convert_to_tensor=True)
    text_embs = model.encode(text_blocks, convert_to_tensor=True)

    similarities = util.cos_sim(graph_emb, text_embs)[0]
    top_idx = int(similarities.argmax())
    top_text = text_blocks[top_idx]
    sim_score = float(similarities[top_idx])

    return {
        "related_text": top_text,
        "similarity_score": round(sim_score, 4),
        "summary": f"The most related text is: '{top_text}'"
    }


In [ ]:
from langchain.tools import tool

@tool
def text_linking_tool(inputs: dict) -> dict:
    """
    Link a graph or table to the most related surrounding text.
    Expects {"graph_summary": "...", "text_blocks": [...]}
    """
    return link_text_to_graph(inputs["graph_summary"], inputs["text_blocks"])


In [ ]:
from langchain.agents import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

text_linking_agent = create_react_agent(
    model,
    tools=[text_linking_tool],
    prompt="""
You are a Text Linking Agent. You are given:
- A summary of a graph or table
- A set of text blocks from a research poster

Your job is to:
- Find the most relevant text block that explains or supports the visual
- Provide a natural language explanation of the link

Use the tool `text_linking_tool` to perform the match.
""",
    name="TextLinkingAgent"
)


# Analysis Summary Agent

In [ ]:
def analysis_summary_tool(inputs: dict) -> str:
    """
    Generate a unified summary from graph and table insights and linked text.
    Expects: {"graphs": [...], "tables": [...]}
    """
    return summarize_analysis(inputs["graphs"], inputs["tables"])

In [ ]:
summary_agent = create_react_agent(
    model,
    tools=[analysis_summary_tool],
    prompt="""
You are a Summary Agent. Your job is to synthesize all the extracted insights from a research poster.
You will be given:
- Graph summaries with related text
- Table summaries with related text

Your task is to write a natural language paragraph summarizing the key findings from the poster.
Use the `analysis_summary_tool` to perform the final summarization.
""",
    name="SummaryAgent"
)

## Integration

In [ ]:
from langgraph.prebuilt import create_swarm
from langgraph.checkpoint import InMemorySaver

checkpointer = InMemorySaver()

workflow = create_swarm(
    agents=[
        layout_agent,
        graph_agent,
        table_agent,
        text_linking_agent,
        summary_agent
    ],
    default_active_agent="LayoutAgent"  # 시작 에이전트
)


In [ ]:
app = workflow.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": "research-poster-1"}}